In [5]:
from os import path
import pandas as pd
from IPython.display import display
from matplotlib import pyplot as plt
from skimage import io
import numpy as np 
import sys 
from aicsimageio import AICSImage
import time 

sys.path.append('../src/')

from extract_pixel_data import extract_pixels_data_variable_bd, voxel_sum_fixed_bd

plt.rcParams["font.family"] = ""

In [2]:
file_path = "/Users/apple/Desktop/Akamatsu_Lab/Data/full_movie_output/track_df_c3_cleaned.pkl"
track_df = pd.read_pickle(file_path)

In [3]:
track_df

,frame,index,amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z,frame_y,tree_id,track_id
0,0,0,175.000000,19.0,1813.0,3.0,2.0,1.0,2.0,0,0,0
1,0,1,188.000000,24.0,1804.0,4.0,2.0,2.0,3.0,0,1,1
2,0,2,182.305173,49.0,569.0,4.0,2.0,1.0,2.0,0,2,2
3,0,3,176.000000,134.0,739.0,4.0,2.0,2.0,3.0,0,3,3
4,0,4,178.000000,39.0,1124.0,4.0,3.0,2.0,3.0,0,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...
515930,129,3059,227.867919,231.0,1921.0,110.0,2.0,2.0,2.0,129,198325,198325
515931,129,3060,269.333333,258.0,1272.0,110.0,2.0,1.0,2.0,129,198326,198326
515932,129,3061,186.475768,259.0,1339.0,110.0,3.0,2.0,2.0,129,198327,198327
515933,129,3062,184.892993,275.0,1325.0,110.0,2.0,2.0,2.0,129,195783,195783


In [4]:
input_file_path = '/Users/apple/Desktop/Akamatsu_Lab/Data/Channel3_full.ome.tiff'


# Load the file
img = AICSImage(input_file_path)

In [6]:
img.dask_data

dask.array<transpose, shape=(130, 1, 116, 2052, 340), dtype=uint16, chunksize=(1, 1, 116, 2052, 340), chunktype=numpy.ndarray>

In [8]:
start_time = time.time()
voxel_sum_array, _ = voxel_sum_fixed_bd(dataframe = track_df,col_names = ['mu_z', 'mu_y', 'mu_x'],
                                     raw_image = img, radii = [4,2,2], frame_col_name = 'frame')
end_time = time.time()
print('time taken (seconds)', end_time - start_time)

current frame is 0
current frame is 1
current frame is 2
current frame is 3
current frame is 4
current frame is 5
current frame is 6
current frame is 7
current frame is 8
current frame is 9
current frame is 10
current frame is 11
current frame is 12
current frame is 13
current frame is 14
current frame is 15
current frame is 16
current frame is 17
current frame is 18
current frame is 19
current frame is 20
current frame is 21
current frame is 22
current frame is 23
current frame is 24
current frame is 25
current frame is 26
current frame is 27
current frame is 28
current frame is 29
current frame is 30
current frame is 31
current frame is 32
current frame is 33
current frame is 34
current frame is 35
current frame is 36
current frame is 37
current frame is 38
current frame is 39
current frame is 40
current frame is 41
current frame is 42
current frame is 43
current frame is 44
current frame is 45
current frame is 46
current frame is 47
current frame is 48
current frame is 49
current fr

In [10]:
start_time = time.time()
offset = [0,0]
col_names = ['mu_z', 'mu_y', 'mu_x']
radi_list = ['sigma_z', 'sigma_y', 'sigma_x']
mean,maximum,minimum,pixel_values,max_loc = extract_pixels_data_variable_bd(raw_image = img, 
        mean_col_names = col_names, dataframe = track_df, radi_col_names = radi_list, frame_col_name = 'frame')

end_time = time.time()
print('time taken (seconds)', end_time - start_time)

current frame number is 0
current frame number is 1
current frame number is 2
current frame number is 3
current frame number is 4
current frame number is 5
current frame number is 6
current frame number is 7
current frame number is 8
current frame number is 9
current frame number is 10
current frame number is 11
current frame number is 12
current frame number is 13
current frame number is 14
current frame number is 15
current frame number is 16
current frame number is 17
current frame number is 18
current frame number is 19
current frame number is 20
current frame number is 21
current frame number is 22
current frame number is 23
current frame number is 24
current frame number is 25
current frame number is 26
current frame number is 27
current frame number is 28
current frame number is 29
current frame number is 30
current frame number is 31
current frame number is 32
current frame number is 33
current frame number is 34
current frame number is 35
current frame number is 36
current fra

In [11]:
track_df['c3_mean_amp'] = mean
track_df['c3_voxel_sum'] = voxel_sum_array
track_df['c3_peak_amp'] = maximum 

In [12]:
track_df.head()

,frame,index,amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z,frame_y,tree_id,track_id,c3_mean_amp,c3_voxel_sum,c3_peak_amp
0,0,0,175.000000,19.0,1813.0,3.0,2.0,1.0,2.0,0,0,0,134.866667,23859,180
1,0,1,188.000000,24.0,1804.0,4.0,2.0,2.0,3.0,0,1,1,143.097143,28894,193
2,0,2,182.305173,49.0,569.0,4.0,2.0,1.0,2.0,0,2,2,132.306667,25547,187
3,0,3,176.000000,134.0,739.0,4.0,2.0,2.0,3.0,0,3,3,141.000000,28280,181
4,0,4,178.000000,39.0,1124.0,4.0,3.0,2.0,3.0,0,4,4,133.453061,26749,183


In [13]:
file_path = "/Users/apple/Desktop/Akamatsu_Lab/Data/full_movie_output/track_df_c3_cleaned_with_c3_vals.pkl"
track_df.to_pickle(file_path)

In [14]:
input_file_path = '/Users/apple/Desktop/Akamatsu_Lab/Data/Channel2_full.ome.tiff'

# Load the file
img_2 = AICSImage(input_file_path)

In [15]:
start_time = time.time()
offset = [0,0]
col_names = ['mu_z', 'mu_y', 'mu_x']
radi_list = ['sigma_z', 'sigma_y', 'sigma_x']
mean,maximum,minimum,pixel_values,max_loc = extract_pixels_data_variable_bd(raw_image = img_2, 
        mean_col_names = col_names, dataframe = track_df, radi_col_names = radi_list, frame_col_name = 'frame')

end_time = time.time()
print('time taken (seconds)', end_time - start_time)

current frame number is 0
current frame number is 1
current frame number is 2
current frame number is 3
current frame number is 4
current frame number is 5
current frame number is 6
current frame number is 7
current frame number is 8
current frame number is 9
current frame number is 10
current frame number is 11
current frame number is 12
current frame number is 13
current frame number is 14
current frame number is 15
current frame number is 16
current frame number is 17
current frame number is 18
current frame number is 19
current frame number is 20
current frame number is 21
current frame number is 22
current frame number is 23
current frame number is 24
current frame number is 25
current frame number is 26
current frame number is 27
current frame number is 28
current frame number is 29
current frame number is 30
current frame number is 31
current frame number is 32
current frame number is 33
current frame number is 34
current frame number is 35
current frame number is 36
current fra

In [16]:
max_loc = np.array(max_loc)

In [17]:
track_df['c2_amp'] = mean
track_df['c2_peak'] = maximum
track_df['c2_peak_x'] = max_loc[:,2]
track_df['c2_peak_y'] = max_loc[:,1]
track_df['c2_peak_z'] = max_loc[:,0]

In [18]:
track_df

,frame,index,amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z,frame_y,tree_id,track_id,c3_mean_amp,c3_voxel_sum,c3_peak_amp,c2_amp,c2_peak,c2_peak_x,c2_peak_y,c2_peak_z
0,0,0,175.000000,19.0,1813.0,3.0,2.0,1.0,2.0,0,0,0,134.866667,23859,180,112.226667,131,21,1813,5
1,0,1,188.000000,24.0,1804.0,4.0,2.0,2.0,3.0,0,1,1,143.097143,28894,193,114.851429,142,25,1806,5
2,0,2,182.305173,49.0,569.0,4.0,2.0,1.0,2.0,0,2,2,132.306667,25547,187,106.986667,123,50,570,5
3,0,3,176.000000,134.0,739.0,4.0,2.0,2.0,3.0,0,3,3,141.000000,28280,181,120.811429,148,135,739,5
4,0,4,178.000000,39.0,1124.0,4.0,3.0,2.0,3.0,0,4,4,133.453061,26749,183,116.130612,137,40,1126,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515930,129,3059,227.867919,231.0,1921.0,110.0,2.0,2.0,2.0,129,198325,198325,189.560000,26412,243,133.184000,187,233,1923,110
515931,129,3060,269.333333,258.0,1272.0,110.0,2.0,1.0,2.0,129,198326,198326,245.466667,31098,309,139.226667,185,260,1273,108
515932,129,3061,186.475768,259.0,1339.0,110.0,3.0,2.0,2.0,129,198327,198327,156.588571,26338,197,127.994286,164,262,1339,109
515933,129,3062,184.892993,275.0,1325.0,110.0,2.0,2.0,2.0,129,195783,195783,151.104000,25350,179,120.080000,146,277,1326,109


In [20]:
col_names = ['c2_peak_z', 'c2_peak_y', 'c2_peak_x']
radi_list = ['sigma_z', 'sigma_y', 'sigma_x']
peak_mean,maxima,_,_,_ = extract_pixels_data_variable_bd(raw_image = img_2, mean_col_names = col_names, 
                                                      dataframe = track_df, radi_col_names = radi_list, 
                                                      frame_col_name = 'frame')

current frame number is 0
current frame number is 1
current frame number is 2
current frame number is 3
current frame number is 4
current frame number is 5
current frame number is 6
current frame number is 7
current frame number is 8
current frame number is 9
current frame number is 10
current frame number is 11
current frame number is 12
current frame number is 13
current frame number is 14
current frame number is 15
current frame number is 16
current frame number is 17
current frame number is 18
current frame number is 19
current frame number is 20
current frame number is 21
current frame number is 22
current frame number is 23
current frame number is 24
current frame number is 25
current frame number is 26
current frame number is 27
current frame number is 28
current frame number is 29
current frame number is 30
current frame number is 31
current frame number is 32
current frame number is 33
current frame number is 34
current frame number is 35
current frame number is 36
current fra

In [21]:
track_df['c2_peak_mean'] = peak_mean

In [22]:
track_df.head()

,frame,index,amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z,frame_y,...,track_id,c3_mean_amp,c3_voxel_sum,c3_peak_amp,c2_amp,c2_peak,c2_peak_x,c2_peak_y,c2_peak_z,c2_peak_mean
0,0,0,175.000000,19.0,1813.0,3.0,2.0,1.0,2.0,0,...,0,134.866667,23859,180,112.226667,131,21,1813,5,119.653333
1,0,1,188.000000,24.0,1804.0,4.0,2.0,2.0,3.0,0,...,1,143.097143,28894,193,114.851429,142,25,1806,5,115.931429
2,0,2,182.305173,49.0,569.0,4.0,2.0,1.0,2.0,0,...,2,132.306667,25547,187,106.986667,123,50,570,5,108.906667
3,0,3,176.000000,134.0,739.0,4.0,2.0,2.0,3.0,0,...,3,141.000000,28280,181,120.811429,148,135,739,5,119.777143
4,0,4,178.000000,39.0,1124.0,4.0,3.0,2.0,3.0,0,...,4,133.453061,26749,183,116.130612,137,40,1126,5,117.432653


In [25]:
voxel_sum_array, _ = voxel_sum_fixed_bd(dataframe = track_df,col_names = ['c2_peak_z', 'c2_peak_y', 'c2_peak_x'],
                                     raw_image = img_2, radii = [4,2,2], frame_col_name = 'frame')


current frame is 0
current frame is 1
current frame is 2
current frame is 3
current frame is 4
current frame is 5
current frame is 6
current frame is 7
current frame is 8
current frame is 9
current frame is 10
current frame is 11
current frame is 12
current frame is 13
current frame is 14
current frame is 15
current frame is 16
current frame is 17
current frame is 18
current frame is 19
current frame is 20
current frame is 21
current frame is 22
current frame is 23
current frame is 24
current frame is 25
current frame is 26
current frame is 27
current frame is 28
current frame is 29
current frame is 30
current frame is 31
current frame is 32
current frame is 33
current frame is 34
current frame is 35
current frame is 36
current frame is 37
current frame is 38
current frame is 39
current frame is 40
current frame is 41
current frame is 42
current frame is 43
current frame is 44
current frame is 45
current frame is 46
current frame is 47
current frame is 48
current frame is 49
current fr

In [26]:
#calculated around the peak value coordinates
track_df['c2_voxel_sum'] = voxel_sum_array

In [27]:
track_df.head()

,frame,index,amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z,frame_y,...,c3_mean_amp,c3_voxel_sum,c3_peak_amp,c2_amp,c2_peak,c2_peak_x,c2_peak_y,c2_peak_z,c2_peak_mean,c2_voxel_sum
0,0,0,175.000000,19.0,1813.0,3.0,2.0,1.0,2.0,0,...,134.866667,23859,180,112.226667,131,21,1813,5,119.653333,26661
1,0,1,188.000000,24.0,1804.0,4.0,2.0,2.0,3.0,0,...,143.097143,28894,193,114.851429,142,25,1806,5,115.931429,26147
2,0,2,182.305173,49.0,569.0,4.0,2.0,1.0,2.0,0,...,132.306667,25547,187,106.986667,123,50,570,5,108.906667,24234
3,0,3,176.000000,134.0,739.0,4.0,2.0,2.0,3.0,0,...,141.000000,28280,181,120.811429,148,135,739,5,119.777143,26840
4,0,4,178.000000,39.0,1124.0,4.0,3.0,2.0,3.0,0,...,133.453061,26749,183,116.130612,137,40,1126,5,117.432653,26159


In [28]:
file_path = "/Users/apple/Desktop/Akamatsu_Lab/Data/full_movie_output/track_df_c3_cleaned_with_c3_c2_vals.pkl"
track_df.to_pickle(file_path)

In [29]:
input_file_path = '/Users/apple/Desktop/Akamatsu_Lab/Data/Channel1_full.ome.tiff'

# Load the file
img_1 = AICSImage(input_file_path)

In [30]:
offset = [0,0]
col_names = ['mu_z', 'mu_y', 'mu_x']
radi_list = ['sigma_z', 'sigma_y', 'sigma_x']
mean,maximum,minimum,pixel_values,max_loc = extract_pixels_data_variable_bd(raw_image = img_1, 
        mean_col_names = col_names, dataframe = track_df, radi_col_names = radi_list, frame_col_name = 'frame')

current frame number is 0
current frame number is 1
current frame number is 2
current frame number is 3
current frame number is 4
current frame number is 5
current frame number is 6
current frame number is 7
current frame number is 8
current frame number is 9
current frame number is 10
current frame number is 11
current frame number is 12
current frame number is 13
current frame number is 14
current frame number is 15
current frame number is 16
current frame number is 17
current frame number is 18
current frame number is 19
current frame number is 20
current frame number is 21
current frame number is 22
current frame number is 23
current frame number is 24
current frame number is 25
current frame number is 26
current frame number is 27
current frame number is 28
current frame number is 29
current frame number is 30
current frame number is 31
current frame number is 32
current frame number is 33
current frame number is 34
current frame number is 35
current frame number is 36
current fra

In [31]:
max_loc = np.array(max_loc)

In [32]:
track_df['c1_amp'] = mean
track_df['c1_peak'] = maximum
track_df['c1_peak_x'] = max_loc[:,2]
track_df['c1_peak_y'] = max_loc[:,1]
track_df['c1_peak_z'] = max_loc[:,0]

In [33]:
voxel_sum_array, _ = voxel_sum_fixed_bd(dataframe = track_df,col_names = ['c1_peak_z', 'c1_peak_y', 'c1_peak_x'],
                                     raw_image = img_1, radii = [4,2,2], frame_col_name = 'frame')


current frame is 0
current frame is 1
current frame is 2
current frame is 3
current frame is 4
current frame is 5
current frame is 6
current frame is 7
current frame is 8
current frame is 9
current frame is 10
current frame is 11
current frame is 12
current frame is 13
current frame is 14
current frame is 15
current frame is 16
current frame is 17
current frame is 18
current frame is 19
current frame is 20
current frame is 21
current frame is 22
current frame is 23
current frame is 24
current frame is 25
current frame is 26
current frame is 27
current frame is 28
current frame is 29
current frame is 30
current frame is 31
current frame is 32
current frame is 33
current frame is 34
current frame is 35
current frame is 36
current frame is 37
current frame is 38
current frame is 39
current frame is 40
current frame is 41
current frame is 42
current frame is 43
current frame is 44
current frame is 45
current frame is 46
current frame is 47
current frame is 48
current frame is 49
current fr

In [34]:
#calculated around the peak value coordinates
track_df['c1_voxel_sum'] = voxel_sum_array

In [35]:
file_path = "/Users/apple/Desktop/Akamatsu_Lab/Data/full_movie_output/track_df_cleaned_final.pkl"
track_df.to_pickle(file_path)

In [36]:
track_df.head()

,frame,index,amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z,frame_y,...,c2_peak_y,c2_peak_z,c2_peak_mean,c2_voxel_sum,c1_amp,c1_peak,c1_peak_x,c1_peak_y,c1_peak_z,c1_voxel_sum
0,0,0,175.000000,19.0,1813.0,3.0,2.0,1.0,2.0,0,...,1813,5,119.653333,26661,134.720000,156,19,1813,5,31407
1,0,1,188.000000,24.0,1804.0,4.0,2.0,2.0,3.0,0,...,1806,5,115.931429,26147,139.565714,167,24,1806,5,32424
2,0,2,182.305173,49.0,569.0,4.0,2.0,1.0,2.0,0,...,570,5,108.906667,24234,118.173333,138,50,568,4,23626
3,0,3,176.000000,134.0,739.0,4.0,2.0,2.0,3.0,0,...,739,5,119.777143,26840,164.662857,217,132,738,7,42946
4,0,4,178.000000,39.0,1124.0,4.0,3.0,2.0,3.0,0,...,1126,5,117.432653,26159,150.069388,199,40,1124,7,38087


In [39]:
track_df.columns

Index(['frame', 'index', 'amplitude', 'mu_x', 'mu_y', 'mu_z', 'sigma_x',
       'sigma_y', 'sigma_z', 'frame_y', 'tree_id', 'track_id', 'c3_mean_amp',
       'c3_voxel_sum', 'c3_peak_amp', 'c2_amp', 'c2_peak', 'c2_peak_x',
       'c2_peak_y', 'c2_peak_z', 'c2_peak_mean', 'c2_voxel_sum', 'c1_amp',
       'c1_peak', 'c1_peak_x', 'c1_peak_y', 'c1_peak_z', 'c1_voxel_sum'],
      dtype='object')

In [41]:
track_df[['track_id','amplitude', 'c3_peak_amp', 'c2_peak', 'c1_peak']]

,track_id,amplitude,c3_peak_amp,c2_peak,c1_peak
0,0,175.000000,180,131,156
1,1,188.000000,193,142,167
2,2,182.305173,187,123,138
3,3,176.000000,181,148,217
4,4,178.000000,183,137,199
...,...,...,...,...,...
515930,198325,227.867919,243,187,604
515931,198326,269.333333,309,185,992
515932,198327,186.475768,197,164,366
515933,195783,184.892993,179,146,302
